# EXERCISE 1 - BESTAS' PROBLEM

## Imports

In [3]:
import gurobipy as gp
from gurobipy import GRB

## Statement

Bestas Energy is an energy utility company that operates a hybrid power plant consisting
of two main components: a wind turbine and an electrolyzer. The wind turbine generates
W = 200kWh of electricity.

In [4]:
W = 200 # kWh

The company has two options for utilizing this energy:

1. Sell the electricity directly to the grid at a rate of λ
E = 10DKK/kWh. However, the
amount of electricity that can be sold directly to the grid is restricted by the grid
connection capacity, which is limited to E = 100kWh.

In [5]:
pr_E = 10 # Price electricity to grid [dkk/kWh]
E = 100 # Max grid connection capacity [kWh]

2. Convert the electricity into hydrogen and sell it at a rate of λ
H = 15DKK/kWh.
The electrolyzer uses electricity to produce hydrogen with a conversion efficiency of
ρ
H = 0.8 and has a maximum hydrogen production capacity of H = 100kWh.

In [6]:
pr_H = 15 # Price hydrogen [dkk/kWh]
ef_h = 0.8 # Electrolyzer efficiency
H = 100 # Max hydrogen production capacity [kWh]

Bestas wants to optimize its production of electricity and hydrogen to maximize its profits.

## Resolution

### a)

Formulate this decision-making problem as an optimization problem, and detail its
decision variables, objective, and constraints.

Optimisation problem:

$$
  \begin{align}
      \textrm{Opt fun:} \max_{x^E, x^H} \quad &10x^E + 15x^H \\
      \textrm{subject to} \quad &x^E + \frac{x^H}{0.8} \leq 200 \\
      &x^E \leq 100 \\
      &x^H \leq 100 \\
      &x^E \geq 0 \\
      &x^H \geq 0
  \end{align}
$$    

In [10]:
model = gp.Model("Ex1")

#Variables
x_E = model.addVar(name='Electricity')
x_H = model.addVar(name='Hydrogen')

#Constrains
constraint_1 = model.addLConstr(x_E + (x_H/ef_h), GRB.LESS_EQUAL, W)
constraint_2 = model.addLConstr(x_E, GRB.LESS_EQUAL, E)
constraint_3 = model.addLConstr(x_H, GRB.LESS_EQUAL, H)
constraint_4 = model.addLConstr(x_E, GRB.GREATER_EQUAL, 0)
constraint_5 = model.addLConstr(x_H, GRB.GREATER_EQUAL, 0)

#Objective Function
model.setObjective(pr_E*x_E + pr_H*x_H, GRB.MAXIMIZE)

model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: AMD Ryzen 7 7840U with Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x55392056
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4000000e+03   1.500000e+01   0.000000e+00      0s
       1    2.2500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.250000000e+03


In [12]:
if model.status == GRB.OPTIMAL:
    optimal_objective = model.ObjVal
    optimal_x_E = x_E.x
    optimal_x_H = x_H.x
    optimal_dual_1 = constraint_1.Pi
    optimal_dual_2 = constraint_2.Pi
    print(f"optimal objective: {optimal_objective}")
    print(f"optimal value of {x_E.VarName}: {optimal_x_E}")
    print(f"optimal value of {x_H.VarName}: {optimal_x_H}")
else:
    print(f"optimization of {model.ModelName} was not successful")

optimal objective: 2250.0
optimal value of Electricity: 75.0
optimal value of Hydrogen: 100.0


### c)

Formulate the dual of the economic dispatch problem. What do the dual variables
associated with each constraint of the dual problem represent?